# DEVSIM TCAD simulator

[DEVSIM](https://devsim.org/) is an open-source semiconductor device simulator; also see [publication](https://joss.theoj.org/papers/10.21105/joss.03898). 

Some of its features include:
* Sharfetter-Gummel discretization of the electron and hole continuity equations
* DC, transient, small-signal AC, and noise solution algorithms
* Solution of 1D, 2D, and 3D unstructured meshes
* Advanced models for mobility and semiclassical approaches for quantum effects
In addition, it operates with a symbolic model evaluation interface, easily allowing new models and their derivatives to be scripted.

There is an active community over at the [DEVSIM forums](https://forum.devsim.org/).

## Meshing

DEVSIM solves equations on unstructured meshes.
It has a built-in 1D and 2D meshing interface, you can solve carrier distributions in waveguide cross-sections.
It also interfaces with GMSH for arbitrary 2D and 3D meshes, which you can use for semiconductor simulations of gdsfactory components.

## Install DEVSIM 

To install DEVSIM you can run `make devsim` from the gdsfactory repo or use the following script.

In [ ]:
install = False # Set True to install the first time

if install:
    !mkdir ~/devsim
    !wget -P ~/devsim https://github.com/devsim/devsim/releases/download/v2.1.0/devsim_linux_v2.1.0.tgz
    !tar --directory ~/devsim zxvf ~/devsim/devsim_linux_v2.1.0.tgz
    !tar zxvf ~/devsim/devsim_linux_v2.1.0.tgz --directory ~/devsim
    !python ~/devsim/devsim_linux_v2.1.0/install.py
    import sys
    !{sys.executable} -m pip install -e ~/devsim/devsim_linux_v2.1.0/lib # Works in this specific way
    %conda install --yes -c conda-forge pyvista

## DC Drift-diffusion simulation

### Setup

You can setup the simulation by defining a strip waveguide cross-section. 
You can change waveguide geometry (core thickness, slab thickness, core width), doping configuration (dopant level, dopant positions), as well as hyperparameters like adaptive mesh resolution at all the interfaces.

In [ ]:
%%capture
from gdsfactory.simulation.devsim import get_simulation_xsection

nm = 1E-9

c = get_simulation_xsection.PINWaveguide(
    wg_width=500 * nm,
    wg_thickness=220 * nm,
    slab_thickness=90 * nm,
)

# Initialize mesh and solver
c.ddsolver()

You can save the device to a tecplot file named `filename.dat` with `c.save_device(filename=filename.dat)`, and then open with [Paraview](https://www.paraview.org/). 

You can also plot the mesh in the Notebook with the `plot` method. By default it shows the geometry.
You can also pass a string to `scalars` to plot a field as color over the mesh.
For instance, acceptor concentration and donor concentration for the PN junction.
The method `c.list_fields()` returns the header of the mesh, which lists all possible fields.

In [ ]:
c.list_fields()

In [ ]:
c.plot(scalars='NetDoping')

In [ ]:
c.plot(scalars='Electrons', log_scale=True)

### Solve

We iteratively solve for the self-consistent carrier distribution for 0.5V of applied forward voltage, iterating with 0.1V steps, and then visualize the electron concentration:

In [ ]:
%%capture
# Find a solution with 1V across the junction, ramping by 0.1V steps
c.ramp_voltage(0.5, 0.1)

In [ ]:
c.plot(scalars='Electrons', log_scale=True)

and similarly for reverse-bias: 

In [ ]:
%%capture
c.ramp_voltage(-0.5, -0.1)

In [ ]:
c.plot(scalars='Electrons', log_scale=True)